In [1]:
import datetime as dt
import os
import re

from enum import Enum
from functools import partial

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDatacenter as dmdc

In [2]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

In [3]:
domo_versions = await dmdc.DomoDatacenter.search_codeengine(auth=token_auth)

# remove codeengine packages that don't parse
domo_versions = [
    domo_version for domo_version in domo_versions if domo_version.functions
]
domo_versions

warning this token has not been validated by who_am_i, run get_auth_token first
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object


[DomoCodeEngine_Version(package_id='511052c6-d852-4dc4-b3dd-6a7e2115303d', version='1.0.0', language='PYTHON', description=None, createdby_id='1893952720', released_dt=None, configuration={'accountsMapping': [], 'mlModel': []}, createdby=None, accounts_mapping=[], ml_model=[], functions=[DomoCodeEngine_Function(name='greeting', displayName='Greeting', description='', language='PYTHON', code="def greeting(): \n  print(sys.version)\n  print('hi')", code_prefix=None)]),
 DomoCodeEngine_Version(package_id='02b77ae9-fc21-40c7-8a62-d9d735e2db9c', version='2.0.6', language='JAVASCRIPT', description='', createdby_id='1658438246', released_dt=datetime.datetime(2024, 3, 15, 16, 39, 52, 606000, tzinfo=tzlocal()), configuration={'accountsMapping': [], 'mlModel': []}, createdby=None, accounts_mapping=[], ml_model=[], functions=[DomoCodeEngine_Function(name='addOwnerToDataset', displayName='Add Owner To Dataset', description='Define the user as an owner of the specified dataset', language='JAVASCRIP

In [4]:
domo_function = next(
    (
        domo_function
        for domo_version in domo_versions
        for domo_function in domo_version.functions
        if domo_function.language == "JAVASCRIPT" and "url" in domo_function.code
    )
)
domo_function.code

"async function createDatasetTag(dataset, tag) {\n  const url = `/api/data/ui/v3/datasources/${dataset}/tags`;\n\n  try {\n    await codeengine.sendRequest('POST', url, [tag]);\n\n    return true;\n  } catch (e) {\n    throw e;\n  }\n}\n\nclass DatasetPermissionInitiator {\n  static createDatasetOwnerPermission(ownerId) {\n    return {\n      id: ownerId,\n      type: 'USER',\n      accessLevel: 'CO_OWNER',\n    };\n  }\n}\n\n\n"

In [5]:
def extract_is_url(code):
    if not "url" in code:
        return False

    return True


extract_is_url(domo_function.code)

True

In [6]:
def extract_url(code):
    pattern_ls = [r"const url = `(.*?)`;", r"const url = (.*?);\n"]

    for pattern in pattern_ls:
        match = re.search(pattern, code)

        if match:
            return match.group(1)


extract_url(domo_function.code)

'/api/data/ui/v3/datasources/${dataset}/tags'

In [7]:
def extract_method(code):
    pattern = r'\.sendRequest\(\s*["\'](.*?)["\']\s*,'

    match = re.search(pattern, code, re.DOTALL)

    if match:
        return match.group(1).upper()

    else:
        return None


extract_method(domo_function.code)

'POST'

In [8]:
def extract_body(code):

    # pattern = r'\.sendRequest\("[^"\*",\s*[^,]*,\s*([^)]*)\)'
    # pattern = r'\.sendRequest\("[^"]*",\s*[^,]*,\s*([^)]*)\)'
    pattern = r"codeengine\.sendRequest\([^,]*,\s*[^,]*,\s*([^)]*)\)"

    match = re.search(pattern, code, re.DOTALL)

    if match:
        return match.group(1)

    else:
        return None


extract_body(domo_function.code)

'[tag]'

In [9]:
def extract_api_call_params(code):
    return {
        "is_url": extract_is_url(code),
        "method": extract_method(code),
        "url": extract_url(code),
        "body": extract_body(code),
    }


# print(domo_function)
domo_function.code
# extract_api_call_params(domo_function.code)

"async function createDatasetTag(dataset, tag) {\n  const url = `/api/data/ui/v3/datasources/${dataset}/tags`;\n\n  try {\n    await codeengine.sendRequest('POST', url, [tag]);\n\n    return true;\n  } catch (e) {\n    throw e;\n  }\n}\n\nclass DatasetPermissionInitiator {\n  static createDatasetOwnerPermission(ownerId) {\n    return {\n      id: ownerId,\n      type: 'USER',\n      accessLevel: 'CO_OWNER',\n    };\n  }\n}\n\n\n"

In [10]:
def javascript_multiline():
    return "/*\n", "*/\n\n"


def python_multiline():
    return '"""\n', '"""\n\n'


class multiline_factory(Enum):
    PYTHON = partial(python_multiline)
    JAVASCRIPT = partial(javascript_multiline)

# EXPORT FUNCTIONS

In [17]:
import domolibrary.classes.DomoCodeEngine as dmce
import domolibrary.utils.chunk_execution as ce


async def handle_export_function(domo_version, domo_function):

    domo_package = await dmce.DomoCodeEngine_Package.get_by_id(
        auth=domo_version.auth, package_id=domo_version.package_id
    )
    domo_version.name = domo_package.name

    output_folder = f"EXPORT/_code_engine/{domo_version.name}/{domo_function.version}/"

    file_name = domo_function.export(
        output_folder=output_folder, file_name=domo_function.name, debug_prn=False
    )

    params = extract_api_call_params(domo_function.code)

    if not params:
        return

    multiline_fn = multiline_factory[domo_function.language].value
    start, end = multiline_fn()

    with open(file_name, "r+") as f:
        content = f.read()
        f.seek(0, 0)

        f.write(start)
        f.writelines([f"{key} - {value}\n" for key, value in params.items() if value])
        f.write(end)

        f.write(content)

    return file_name


await ce.gather_with_concurrency(
    *[
        handle_export_function(domo_version=domo_version, domo_function=domo_function)
        for domo_version in domo_versions
        for domo_function in domo_version.functions
    ],
    n=10,
)

['EXPORT/code_engine/python/1.0.0/greeting.py',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/addOwnerToDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/addOwnersToDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/appendToDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/createDatasetTag.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getDatasetOwner.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getFirstMatch.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getMetadata.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getNumberOfCardsPoweredByDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getNumberOfDataflowsPoweredByDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/getValueFor.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/queryWithSql.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/removeUserFromDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/removeUsersFromDataset.js',
 'EXPORT/code_engine/DOMO DataSets/2.0.6/revokeDatasetAccess.js',
 'EXPORT/code_engine/DOMO DataSets/2

In [ ]:
dt.datetime.now().strftime("%H:%M")

'07:00'